In [1]:
import openai
import datetime
from utils.textprocessing_helpers import preprocess

In [ ]:
openai.api_base = "http://localhost:8000/v1"
openai.api_key = 'EMPTY'
model = 'train_solar'
# model = "train_phi2"
# model = 'train_llama2_13b_chat'
# model = 'train_mistral7b'
# model = 'train_zephyr7b-beta'

In [ ]:
# Run this command in a terminal

# CUDA_VISIBLE_DEVICES=0,1 python src/api_demo.py \
#     --model_name_or_path upstage/SOLAR-10.7B-Instruct-v1.0 \
#     --adapter_name_or_path saves/SOLAR-10.7B-Chat/lora/train_solar  \
#     --template solar \
#     --finetuning_type lora &

In [ ]:
# CUDA_VISIBLE_DEVICES=0,1 python src/api_demo.py \
#     --model_name_or_path mistralai/Mistral-7B-Instruct-v0.1 \
#     --adapter_name_or_path saves/Mistral-7B-Chat/lora/train_mistral7b \
#     --template mistral \
#     --finetuning_type lora &


In [ ]:
# CUDA_VISIBLE_DEVICES=0,1 python src/api_demo.py \
#     --model_name_or_path meta-llama/Llama-2-13b-chat-hf \
#     --adapter_name_or_path saves/LLaMA2-13B-Chat/lora/train_llama2_13b_chat \
#     --template llama2 \
#     --finetuning_type lora &


In [ ]:
# CUDA_VISIBLE_DEVICES=0,1 python src/api_demo.py \
#     --model_name_or_path HuggingFaceH4/zephyr-7b-beta \
#     --adapter_name_or_path saves/Zephyr-7B-Beta-Chat/lora/train_zephyr7b-beta \
#     --template zephyr \
#     --finetuning_type lora &


In [ ]:
# CUDA_VISIBLE_DEVICES=0,1 python src/api_demo.py \
#     --model_name_or_path upstage/SOLAR-10.7B-Instruct-v1.0 \
#     --adapter_name_or_path saves/SOLAR-10.7B-Chat/lora/train_solar  \
#     --template solar \
#     --finetuning_type lora &

In [4]:
def log_message(message, log_file=f'logs/test_{model}_log.txt'):
    timestamp = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    formatted_message = f"[{timestamp}] {message}\n"
    
    with open(log_file, 'a') as file:
        file.write(formatted_message)
    return

In [3]:
def query_model(prompt):
    generated_text = ''    
    response = openai.ChatCompletion.create(
        model="main",
        messages=[
            {"role": "user", "content": f'{prompt}'}
        ],
        temperature=0.2,
        max_new_tokens=1024,
        stream=True
    )
    for chunk in response:
        if hasattr(chunk.choices[0].delta, "content"):
            # print(chunk.choices[0].delta.content, end="", flush=True)
            generated_text = generated_text + chunk.choices[0].delta.content
    return generated_text

In [21]:
def query_model_no_stream(prompt):
        
    response = openai.ChatCompletion.create(
        model="main",
        messages=[
            {"role": "user", "content": f'{prompt}'}
        ],
        temperature=0.2,
        max_new_tokens=1024
    )
    generated_text = response['choices'][0]['message']['content']
    return generated_text

In [6]:
prompt_instruction = '''Analyze text against provided triplets, classifying claims as "Attributable", "Contradictory", or "Exploratory".
Justify your classification using the following structure:
- "text_span": Text under evaluation.
- "prediction": Category of the text (Attributable/Contradictory/Exploratory).
- "triplets": Relevant triplets (if any, else "NA").
- "rationale": Reason for classification.

For multiple claims, number each component (e.g., "text_span1", "prediction1"). Use "NA" for inapplicable keys.

Example:
"text_span1": "Specific claim",
"prediction1": "Attributable/Contradictory/Exploratory",
"triplets1": "Relevant triplets",
"rationale1": "Prediction justification",
...

Input for analysis:
'''

In [ ]:
def gen_prompt(text, triplets):
    text_req = f'''
    - Text: {text}
    - Triplets: {triplets}
    '''
    prompt = prompt_instruction+text_req
    return prompt

In [13]:
input_text = 'Barack Obama was educated at Harvard, and he signed the Affordable Care Act.'

In [ ]:
paths, wikients = preprocess(input_text)
prompt = gen_prompt(input_text, paths)
response = query_model_no_stream(prompt)
print(response)